# Face Mask Detection Project

# Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, optimizers, models
from keras.layers import Conv2D, BatchNormalization, PReLU, MaxPooling2D
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Face Dataset.zip", 'r')
zip_ref.extractall("/content/drive/MyDrive")
zip_ref.close()

# Load Data

In [ ]:
data_path="/content/drive/MyDrive/Face Dataset"

In [ ]:
train_data_path=os.path.join(data_path, 'Train Data')
val_data_path=os.path.join(data_path, 'Validation Data')
test_data_path=os.path.join(data_path, "Test Data")

In [ ]:
names=os.listdir(train_data_path)
names

In [ ]:
def total_img(path,names):
    count=0
    for name in names:
        size=len(os.listdir(os.path.join(path, name)))
        count+=size
    return count

In [ ]:
print("Total Training Images: ", total_img(train_data_path, names))
print("Total Validation Images: ", total_img(val_data_path, names))
print("Total Test Images: ", total_img(test_data_path, names))

# Data preprocessing

# Using ImageDataGenerator to read images from directories

In [ ]:
datagenerator=ImageDataGenerator( 
                                   rescale=1./255,
                                )

In [ ]:
train_gen= datagenerator.flow_from_directory(
                                              train_data_path,
                                              batch_size=39,
                                              target_size=(56,56),
                                              class_mode='binary'
                                            )

In [ ]:
val_gen= datagenerator.flow_from_directory(
                                            val_data_path,
                                            target_size=(56,56),
                                            batch_size=26,
                                            class_mode='binary'
                                        )

In [ ]:
labels=train_gen.class_indices
labels

# Network Architecture

In [ ]:
from keras.layers import Input, Conv2D, Add, Activation

def normal_cell(x, n_filters, n_inputs, n_operations, n_outputs):
    inputs = [x]
    for i in range(n_inputs):
        inputs.append(Input(shape=(None, None, n_filters)))

    operations = []
    for i in range(n_operations):
        operation = Conv2D(filters=n_filters, kernel_size=3, strides=1, padding='same')(x)
        operations.append(operation)

    x = Add()(operations)
    x = Activation('relu')(x)

    return x

In [ ]:
from keras.layers import Input, MaxPooling2D, Conv2D, Concatenate

def reduction_cell(x, n_filters):
    # apply max pooling with stride 2
    x = MaxPooling2D(strides=2, padding='same')(x)
    
    # apply 1x1 convolution with n_filters
    c1 = Conv2D(n_filters, kernel_size=1, activation='relu', padding='same')(x)
    
    # apply 3x3 convolution with n_filters
    c2 = Conv2D(n_filters, kernel_size=3, activation='relu', padding='same')(x)
    
    # concatenate the outputs of the 1x1 and 3x3 convolution
    x = Concatenate()([c1, c2])
    return x

In [ ]:

# Input tensor
inputs = Input(shape=(56, 56, 3))

# 2D convolutional layer
x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(inputs)
x = BatchNormalization()(x)

# Normal cell
x = normal_cell(x, n_filters=128, n_inputs=3, n_operations=6, n_outputs=1)

# Reduction cell
x = reduction_cell(x, n_filters=256)

# Normal cell
x = normal_cell(x, n_filters=256, n_inputs=7, n_operations=11, n_outputs=2)

# Reduction cell
x = reduction_cell(x, n_filters=512)

# Normal cell
x = normal_cell(x, n_filters=512, n_inputs=12, n_operations=15, n_outputs=1)

# Reduction cell
x = reduction_cell(x, n_filters=1024)

# PReLU activation and convolutional layer
x = PReLU()(x)
x = Conv2D(512, (1, 1), padding='same')(x)
x = BatchNormalization()(x)
x = PReLU()(x)
# Final convolutional layers
x = Conv2D(512, (7, 7), padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (1, 1), padding='same')(x)
x = BatchNormalization()(x)

x=layers.Flatten()(x)
x=layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs, x)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Training

In [ ]:
history=model.fit(train_gen, epochs=15, validation_data=val_gen)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs= range(1,len(loss)+1)
plt.plot(epochs, loss, 'bo', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss" )
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/Face Mask Detection Project/face detection new model.h5")

In [ ]:
testdatagen=ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator=testdatagen.flow_from_directory(
                                            test_data_path,
                                            batch_size=12,
                                            target_size=(56,56),
                                            class_mode='binary'
                                            )

In [ ]:
loss,acc= model.evaluate(test_generator)

In [ ]:
print("Test Data Loss: ",loss)
print("Test Data Accuracy: ",acc)